In [1]:
import syft as sy
import pandas as pd
import numpy as np
from nacl.encoding import HexEncoder
from nacl.signing import SigningKey
from syft.core.adp.entity import DataSubject

Seeding DB with roles!


In [2]:
data = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv")

# Get canada trade data and convert into unit millions
canada_trade = (np.array(list(data['Trade Value (US$)'])) / 1000000)[0:10]
trade_partners = ((list(data['Partner'])))[0:10]

data_batch = canada_trade
data_batch = (data_batch * 10).astype('int32')

entities = list()
for i in range(len(data_batch)):
    entities.append(DataSubject(name=trade_partners[i]))

# Upload a private dataset to the Domain object, as the root owner
x = sy.Tensor(data_batch)#.private(0, 2, entity=entities[0]).tag("trade_flow")
x.public_shape = x.shape

y = sy.Tensor(data_batch)#.private(0, 2, entity=entities[0]).tag("trade_flow")
y.public_shape = y.shape

/Users/andrewliamtrask/opt/anaconda3/envs/syft/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def encode_key(key: SigningKey) -> str:
    return key.encode(encoder=HexEncoder).decode("utf-8")

_alice = sy.Domain("alice")
alice = _alice.get_root_client()

alice_key = SigningKey.generate()
_alice.users.signup(
    name="Bob",
    email="bob@gmail.com",
    password="letmein",
    budget=10000000000,
    role=1,
    private_key=encode_key(alice_key),
    verify_key=encode_key(alice_key.verify_key),
)
alice.verify_key = alice_key.verify_key
alice.signing_key = alice_key

_bob = sy.Domain("bob")
bob = _bob.get_root_client()

bob_key = SigningKey.generate()
_bob.users.signup(
    name="Bob",
    email="bob@gmail.com",
    password="letmein",
    budget=10000000000,
    role=1,
    private_key=encode_key(bob_key),
    verify_key=encode_key(bob_key.verify_key),
)
bob.verify_key = bob_key.verify_key
bob.signing_key = bob_key

Seeding DB with roles!
Setup hasnt run yet so ignoring set_node_uid
Seeding DB with roles!
Setup hasnt run yet so ignoring set_node_uid


In [4]:
xp = x.send(alice)
yp = y.send(bob)

dp_x = x.private(0, 2, entity=entities[0]).tag("trade_flow")

In [5]:
dp_x

Tensor(child=SingleEntityPhiTensor(entity=Other Asia, nes, child=[ 0  1 14  0  0  0  0  3  0  0]))

In [6]:
dp_x_p = dp_x.send(alice)

In [7]:
type(dp_x_p), dp_x_p.id_at_location

(syft.core.tensor.autodp.single_entity_phi.TensorWrappedSingleEntityPhiTensorPointer,
 <UID: 82e220ffa9884ce1b2ee1cc77bceb060>)

In [8]:
out = dp_x_p.share(bob)

[2021-08-27T14:16:37.326952+0100][CRITICAL][logger]][23040] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-08-27T14:16:37.328057+0100][CRITICAL][logger]][23040] 'Class' object has no attribute 'generate_przs_on_dp_tensor'
[2021-08-27T14:16:37.383841+0100][CRITICAL][logger]][23040] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-08-27T14:16:37.384585+0100][CRITICAL][logger]][23040] 'Class' object has no attribute 'generate_przs_on_dp_tensor'


In [9]:
# type(out), [child.id_at_location for child in out.child]

In [10]:
out2 = out + out

In [11]:
out3 = out2.publish(sigma=0.01)

a
b
c


PROCESSING PUBLISH SCALAR ACTION!!!


Verify Key:<class 'nacl.signing.VerifyKey'>
<UID: 884b02cf793f43e29c3aba1ef1c2a44d>
IntermediateGamma.publish
<class 'syft.core.tensor.autodp.initial_gamma.InitialGammaTensor'>
<class 'list'>


Has Budget:True
SPEND:806642.5471077008
USER BUDGET:10000000000.0
d
a
b
c


PROCESSING PUBLISH SCALAR ACTION!!!


Verify Key:<class 'nacl.signing.VerifyKey'>
<UID: 51512571c70e48a1807c82f0d57cffa3>
IntermediateGamma.publish
<class 'syft.core.tensor.autodp.initial_gamma.InitialGammaTensor'>
<class 'list'>


Has Budget:True
SPEND:806642.5471077008
USER BUDGET:10000000000.0
d


In [13]:
alice.privacy_budget

9999193357.452892

In [14]:
bob.privacy_budget

9999193357.452892

In [11]:
# type(out2), [child.id_at_location for child in out.child]

In [12]:
f = out3.get()

In [13]:
type(f)

syft.core.tensor.tensor.Tensor

In [14]:
f

Tensor(child=ShareTensor(child=[ 0  3 28  1  0  0  0  6  0 -1]))

In [15]:
x

Tensor(child=SingleEntityPhiTensor(entity=Other Asia, nes, child=[ 0  1 14  0  0  0  0  3  0  0]))